In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 

import pandas as pd

nav = webdriver.Chrome()

tabela_produtos = pd.read_excel("buscas.xlsx")
display(tabela_produtos)

The chromedriver version (138.0.7204.183) detected in PATH at C:\Users\betin\anaconda3\chromedriver.exe might not be compatible with the detected chrome version (139.0.7258.155); currently, chromedriver 139.0.7258.154 is recommended for chrome 139.*, so it is advised to delete the driver in PATH and retry


,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 15 128gb,mini watch,3000,4500
1,rtx 3060,zota galax,2000,4500


In [4]:
import time

def busca_google_shopping(nav, produto, termos_banidos, preco_minimo, preco_maximo):
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(' ')
    lista_termos_nome_produtos = produto.split(' ')


    lista_ofertas = []
    
    
    nav.get('https://www.google.com/')
    nav.find_element('xpath', '//*[@id="APjFqb"]').send_keys(produto, Keys.ENTER)
    
    shopping = WebDriverWait(nav, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//a[normalize-space()='Shopping']"))
    )
    shopping.click()
    
    lista_resultados = nav.find_elements(By.CLASS_NAME, 'pla-unit-container')
    
    for resultado in lista_resultados:
        nome = resultado.find_element(By.CLASS_NAME, 'pymv4e').text
        nome = nome.lower()
    
        # verificacao do nome - se no nome tem algum termo banido   
        tem_termos_banidos = False
        for palavra in lista_termos_banidos:
            if palavra in nome:
                tem_termos_banidos = True

         # verificar se no nome tem todos os termos do nome do produto
        tem_todos_termos_produto = True
        for palavra in lista_termos_nome_produtos:
            if palavra not in nome:
                tem_todos_termos_produto = False
                
        if not tem_termos_banidos and tem_todos_termos_produto:
            preco = resultado.find_element(By.CLASS_NAME, 'e10twf').text
            preco = preco.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".")
            preco = float(preco)
        
            preco_minimo = float(preco_minimo)
            preco_maximo = float(preco_maximo)
    
        if preco_minimo <= preco <= preco_maximo:
            link = resultado.find_element(By.CLASS_NAME, 'plantl').get_attribute('href')
            lista_ofertas.append((nome, preco, link))
    return lista_ofertas


def busca_buscape(nav, produto, termos_banidos, preco_minimo, preco_maximo):

    preco_maximo = float(preco_maximo)
    preco_minimo = float(preco_minimo)
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(" ")
    lista_termos_produto = produto.split(" ")

    nav.get("https://www.buscape.com.br/")

     # pesquisar pelo produto no buscape
    nav.find_element(By.CLASS_NAME, 'SearchInput_Input__W2vzU').send_keys(produto, Keys.ENTER)
    
    # pegar a lista de resultados da busca do buscape
    time.sleep(5)
    lista_resultados = nav.find_elements(By.CLASS_NAME, 'ProductCard_ProductCard_Inner__gapsh')
    
    # para cada resultado
    lista_ofertas = []
    for resultado in lista_resultados:
        try:
            preco = resultado.find_element(By.CLASS_NAME, 'Text_MobileHeadingS__HEz7L').text
            nome = resultado.find_element(By.CLASS_NAME, 'ProductCard_ProductCard_Name__U_mUQ').text
            nome = nome.lower()
            link = resultado.get_attribute('href')
            
            # verificacao do nome - se no nome tem algum termo banido
            tem_termos_banidos = False
            for palavra in lista_termos_banidos:
                if palavra in nome:
                    tem_termos_banidos = True  
                    
            # verificar se no nome tem todos os termos do nome do produto
            tem_todos_termos_produto = True
            for palavra in lista_termos_produto:
                if palavra not in nome:
                    tem_todos_termos_produto = False            
            
            if not tem_termos_banidos and tem_todos_termos_produto:
                preco = preco.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".")
                preco = float(preco)
                if preco_minimo <= preco <= preco_maximo:
                    lista_ofertas.append((nome, preco, link))
        except:
            pass
    return lista_ofertas
    

In [5]:
tabela_ofertas = pd.DataFrame()

for linha in tabela_produtos.index:
    produto = tabela_produtos.loc[linha, "Nome"]
    termos_banidos = tabela_produtos.loc[linha, "Termos banidos"]
    preco_minimo = tabela_produtos.loc[linha, "Preço mínimo"]
    preco_maximo = tabela_produtos.loc[linha, "Preço máximo"]
    
    lista_ofertas_google_shopping = busca_google_shopping(nav, produto, termos_banidos, preco_minimo, preco_maximo)
    if lista_ofertas_google_shopping:
        tabela_google_shopping = pd.DataFrame(lista_ofertas_google_shopping, columns=['produto', 'preco', 'link'])
        tabela_ofertas = pd.concat([tabela_ofertas, tabela_google_shopping])
    else:
        tabela_google_shopping = None
        
    lista_ofertas_buscape = busca_buscape(nav, produto, termos_banidos, preco_minimo, preco_maximo)
    if lista_ofertas_buscape:
        tabela_buscape = pd.DataFrame(lista_ofertas_buscape, columns=['produto', 'preco', 'link'])
        tabela_ofertas = pd.concat([tabela_ofertas, tabela_buscape])
    else:
        tabela_buscape = None

display(tabela_ofertas)

,produto,preco,link
0,"iphone 15 apple (128gb) preto, tela de 6,1"", 5...",4099.00,https://secure3.fastshop.com.br/iphone-15-appl...
1,"apple iphone 15 128gb preto 6,1"" 48mp ios 5g, ...",4299.00,https://www.magazineluiza.com.br/apple-iphone-...
2,apple iphone 15 (128 gb) — azul,4299.00,https://www.amazon.com.br/Apple-iPhone-15-128-...
3,"apple iphone 15 128gb 6,1 48mp preto",4299.30,https://www.casasbahia.com.br/apple-iphone-15-...
4,"iphone 15 apple (128gb) azul, tela de 6,1"", 5g...",4099.00,https://secure3.fastshop.com.br/iphone-15-appl...
5,apple iphone 15 (256 gb) — preto,4099.00,https://www.amazon.com.br/Apple-iPhone-15-256-...
6,apple iphone 15 (128 gb) - preto - bom (recond...,4099.00,https://www.mercadolivre.com.br/apple-iphone-1...
7,"apple iphone 15 128gb azul 6,1"" 48mp ios 5g, 1...",4299.00,https://www.magazineluiza.com.br/apple-iphone-...
8,"apple iphone 16e 128gb preto 6,1"" 48mp ios 5g,...",4299.00,https://www.magazineluiza.com.br/apple-iphone-...
9,apple iphone 15 (128 gb) — rosa,4299.00,https://www.amazon.com.br/Apple-iPhone-15-128-...


In [6]:
# exportar pro excel
tabela_ofertas = tabela_ofertas.reset_index(drop=True)
tabela_ofertas.to_excel("Ofertas.xlsx", index=False)


In [7]:
# enviar por e-mail o resultado da tabela
import win32com.client as win32

#verificando se existe alguma oferta dentro da tabela de ofertas
if len(tabela_ofertas.index) > 0:
    # vou enviar email
    outlook = win32.Dispatch('outlook.application')
    mail = outlook.CreateItem(0)
    mail.To = 'xxxxxxxxxx@gmail.com'
    mail.Subject = 'Produto(s) Encontrado(s) na faixa de preço desejada'
    mail.HTMLBody = f"""
    <p>Prezados,</p>
    <p>Encontramos alguns produtos em oferta dentro da faixa de preço desejada. Segue tabela com detalhes</p>
    {tabela_ofertas.to_html(index=False)}
    <p>Qualquer dúvida estou à disposição</p>
    <p>Att.,</p>
    """
    
    mail.Send()

nav.quit()  